In [ ]:
import geemap
import ee
import os
ee.Initialize()

In [ ]:
# Choose these coordinates when for the final product

vertices = [
    [
       -78.6788057698592,
        45.68985184684981 
    ],
    [
       -78.67627376454914,
        45.68985184684981 
    ],
    [
        -78.67627376454914,
         45.692621011934015
    ],
    [
        -78.6788057698592,
        45.692621011934015
    ],
    [
        -78.6788057698592,
        45.68985184684981
    ]
]

poly = ee.Geometry.Polygon(vertices)

In [ ]:
# Choose these coordinates when for the loop test

vertices = [
    [
       -78.67816051316345,
        45.687667175820906 
    ],
    [
       -78.67750068974578,
        45.687667175820906 
    ],
    [
        -78.67750068974578,
         45.68808313765678
    ],
    [
        -78.67816051316345,
        45.68808313765678
    ],
    [
        -78.67816051316345,
        45.687667175820906
    ]
]

poly = ee.Geometry.Polygon(vertices)

In [ ]:
vertices = [
    [
      -78.68270923328974,
      45.68524774437538
    ],
    [
     -78.67440511417963,
      45.68524774437538
    ],
    [
      -78.67440511417963,
      45.69305715580808
    ],
    [
      -78.68270923328974,
      45.493210350454774
    ],
    [
      -78.41757353217413,
      45.69305715580808
    ],
    [
      -78.68270923328974,
      45.68524774437538
    ]
]
    
poly = ee.Geometry.Polygon(vertices)

In [ ]:
def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()

def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

In [ ]:
def calcAWEInsh(image):
    AWEInsh = image.expression(
        "4 * (TM2 - TM5) /10000 - (0.25 * TM4 + 2.75 * TM7) /10000", {
            "TM2" : image.select("SR_B2"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("AWEInsh").set({'system:time_start': image.get('system:time_start')})
    return AWEInsh.multiply(1000)

def calcAWEInshL8(image):
    AWEInsh = image.expression(
        "4 * (TM2 - TM5) /10000 - (0.25 * TM4 + 2.75 * TM7) /10000", {
            "TM2" : image.select("SR_B3"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("AWEInsh").set({'system:time_start': image.get('system:time_start')})
    return AWEInsh.multiply(1000)

In [ ]:
filters = [
    ee.Filter.bounds(poly),
    ee.Filter.lt("CLOUD_COVER", 50)
]

l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcAWEInshL8)
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcAWEInsh)
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcAWEInsh)

AWEInsh = ee.ImageCollection(l5.merge(l7).merge(l8)).sort('system:time_start')

In [ ]:
# Since we rescaled the AWEInsh by dividing 10000, we no longer need this ".int()" after ".toBands"
# Otherwise, the resampled dataset may appear to be binary.
img = AWEInsh.toBands().int16()

In [ ]:
### just in case the band names are not kept when the image is exported
### save these ones to rename them offline
bandNames = img.bandNames().getInfo()

if os.path.exists('band_names.txt'):
    os.remove('band_names.txt')
    
with open('band_names.txt', 'w') as file:
    for b in bandNames:
        file.write(f"{b}\n")

In [ ]:
from ee.batch import Export

task = Export.image.toDrive(
    image = img,
    region = poly,
    scale = 30,
    folder = 'gee_exports',
    fileNamePrefix = "AWEInsh_TimeSeries_Final_Product",
    description = "Export AWEInsh time series final product",
    crs = "EPSG:32617"
)

task.start()

In [ ]:
img.getInfo()